![LOGO](img/MODIN_ver2_hrz.png)

<center><h2>Scale your pandas workflows by changing one line of code</h2>


# Exercise 3: Not Implemented and Looping

**GOAL**: Learn what happens when a function is not yet supported in Modin and what functionality is not possible to accelerate

When functionality has not yet been implemented, we default to pandas

![](img/convert_to_pandas.png)

We convert a Modin dataframe to pandas to do the operation, then convert it back once it is finished. These operations will have a high overhead due to the communication involved and will take longer than pandas.

When this is happening, a warning will be given to the user to inform them that this operation will take longer than usual. For example, `DataFrame.kurtosis` is not yet implemented. In this case, when a user tries to use it, they will see this warning:

```
UserWarning: `DataFrame.kurtosis` defaulting to pandas implementation.
```

## Concept for exercise: Default to pandas

In this section of the exercise we will see first-hand how the runtime is affected by operations that are not implemented.

In [ ]:
import modin.pandas as pd
import numpy as np
import time

frame_data = np.random.randint(0, 100, size=(2**12, 2**8))
df = pd.DataFrame(frame_data).add_prefix("col")

In [ ]:
start = time.time()

kurtosis = df.kurtosis()

end = time.time()
print("Modin kurtosis took {} seconds.".format(end - start))

In [ ]:
kurtosis

In [ ]:
type(kurtosis)

## Concept for exercise: Register custom functions

Modin's user-facing API is pandas, but it is possible that we do not yet support your favorite or most-needed functionalities. It is also possible that your user-defined function can be executed more efficiently if you pre-define the behavior or computing paradigm. To solve either case, it is possible to register a custom function to be applied to your data.

### Registering a custom function for all query compilers

To register a custom function for a query compiler, we first need to import it:

```python
from modin.backends.pandas.query_compiler import PandasQueryCompiler
```

The `PandasQueryCompiler` is responsible for defining and compiling the queries that can be operated on by Modin, and is specific to the pandas backend. Any queries defined here must also both be compatible with and result in a `pandas.DataFrame`. Many functionalities are very simply implemented, as you can see in the current code: [Link](https://github.com/modin-project/modin/blob/f15fb8ea776ed039893130b1e85053e875912d4b/modin/backends/pandas/query_compiler.py#L365).